In [1]:
from __future__ import division
from __future__ import print_function  
import warnings
import pandas as pd
import gsp_support as gsp
import matplotlib.pyplot as plt
import numpy as np
pd.options.plotting.backend = "plotly"
import datetime
output_folder = 'FINAL/Disaggregation/stage2_filter/'

In [14]:
l = ['t1','t2','t3','t4','t5','t6','t7']
output = pd.DataFrame()

for s in l:
    csv = pd.read_csv(output_folder+s+'.csv')
    csv['x_Timestamp']= pd.to_datetime(csv['x_Timestamp'])
    csv = csv.set_index('x_Timestamp')
    output = output.append(csv)

In [15]:
ll = list(output['meter'].unique())

In [16]:
survey = pd.read_csv('FINAL- Appliance stock survey and metadata 20Nov19.xlsx - FINAL- Appliance stock survey a.csv')

ac_list = list(survey.loc[survey['q_409_ac_yn']==1]['home_no'])
ac_list = list(set(ac_list) - set(['BR17','BR20','BR42']))

In [18]:
set(ac_list)-set(ll)

set()

In [19]:
output.to_csv(output_folder+'3min_disaggregated_final.csv')

In [22]:
len(output['meter'].unique())

31

In [53]:
# get the new monthly statistics

output_folder = 'FINAL/monthly statistics/'

In [25]:
output['timestamp'] = output.index
output['m-y'] = output['timestamp'].dt.to_period('M')
output = output.drop(['timestamp'],axis=1)

In [27]:
# functions for statistics

def daily_ac_hrs(df):
    num = df.loc[df['ac_on']==1].shape[0]
    hrs = (num*3)/60
    return hrs

def daily_ac_avg_load(df):
    num = df.loc[df['ac_on']==1]['ac_power'].mean()
    return num

def daily_avg_energy(df):
    num = (df['power']/20000).sum()
    return num

def daily_avg_car(df):
    n1 = df.loc[df['final_compressor']==1].shape[0]
    n2 = df.loc[df['ac_on']==1].shape[0]
    if n2>0:
        num = (n1/n2)*100
    else: 
        num = 0
    
    return num

In [52]:
# a dataframe that tells me the number of days for each meters for each for which data is available

# days = {k: [] for k in mm}
my = []
met = []
n = []
nac = []
d_avg_ac_hrs = []
acp = []
per_day_e = []
ty = []
car = []

grouped = output1.groupby('meter')

for name, group in grouped:
    print(name)
    sub = grouped.get_group(name)
    month_list = list(sub['m-y'].unique())
    
    for m in month_list:
        sub_month = sub.loc[sub['m-y']==m]
        s = sub_month.index.date
        ac_ = sub_month.loc[sub_month['ac_on']==1]
        
        num_days_ac = len(list(np.unique(ac_.index.date)))
        num_days = len(list(np.unique(s)))
        hr = sub_month.groupby(sub_month.index.date).apply(daily_ac_hrs)
        hr_ = hr.loc[hr>0]
        if len(hr_)>0:
            dac = sum(hr_)/len(hr_)
        else:
            dac = np.nan
            
        ac_power = sub_month.loc[sub_month['ac_on']==1]['ac_power'].mean() #baad me dekh lena
        
        avg_e = sub_month.groupby(sub_month.index.date).apply(daily_avg_energy).mean()
        
        row = survey.loc[survey['home_no']==name]
        if row['q_411_ac1_type'].values[0]==1:
            typee = 'Window '
        elif row['q_411_ac1_type'].values[0]==2:
            typee = 'Split '
        else:
            typee = 'Inverter'
        c = sub_month.groupby(sub_month.index.date).apply(daily_avg_car)
        c_ = c.loc[c>0]    
        cc = c_.mean()        
        met.append(name)
        my.append(str(m))
        n.append(num_days)
        nac.append(num_days_ac)
        d_avg_ac_hrs.append(dac)
        acp.append(ac_power)
        per_day_e.append(avg_e)
        ty.append(typee)
        car.append(cc)
        
        
        
        
       


BR06
BR07
BR08
BR11
BR12
BR13
BR18
BR22
BR23
BR24
BR31
BR35
BR36
BR38
BR45
BR49
BR51
MH02
MH07
MH10
MH14
MH18
MH21
MH22
MH23
MH24
MH27
MH33
MH36
MH43
MH45


In [62]:
num_days = pd.DataFrame({'Meter':met, 'M-Y':my, 'num_days_data_available':n,'ac_on_days':nac, 'd_avg_ac_hrs':d_avg_ac_hrs,'average ac load':acp,'daily_avg_consumption':per_day_e,'ac type':ty})

In [63]:
#rounding off and extra columns

num_days['perc_days_ac_on'] = np.round((num_days['ac_on_days']/num_days['num_days_data_available'])*100,2)
num_days['d_avg_ac_hrs'] = num_days['d_avg_ac_hrs']
num_days['monthly ac usage hrs'] = np.round(num_days['d_avg_ac_hrs']*(num_days['perc_days_ac_on']/100)*30,2)
num_days['average ac load'] = np.round(num_days['average ac load'])
num_days['daily_avg_consumption'] = num_days['daily_avg_consumption']

In [64]:
# total monthly energy consumption by AC


num_days['monthly ac consumption'] = np.round((num_days['average ac load']/1000)*num_days['monthly ac usage hrs'],2)
num_days['monthly consumption'] = np.round((num_days['daily_avg_consumption']*30),2)


In [66]:
num_days.to_csv(output_folder+'monthly3.csv')

In [46]:
start_date = '2019-05-01' # from may
end_date = '2019-11-30'  # till september
mask = (output.index >= start_date) & (output.index < end_date)
start_date1 = '2020-03-01' # from may
end_date1 = '2020-11-30'  # till september
mask1 = (output.index >= start_date1) & (output.index < end_date1)

In [47]:
output1 = output.loc[mask | mask1]

In [49]:
output1['m-y'].unique()

<PeriodArray>
['2019-05', '2019-06', '2019-07', '2019-08', '2019-09', '2019-10', '2019-11',
 '2020-03', '2020-04', '2020-05', '2020-06', '2020-07', '2020-08', '2020-09',
 '2020-10', '2020-11']
Length: 16, dtype: period[M]

In [73]:
output_folder = 'FINAL/Disaggregation/stage2_filter/'


In [74]:
output1.to_csv(output_folder+'3 min disaggregated result FINAL with removed meters.csv')

In [68]:
output1 = output1.loc[~output1['meter'].isin(['BR35','BR36','BR49','BR13'])]

In [ ]:
# daily logs

In [72]:
output1['meter'].unique()

array(['MH24', 'MH27', 'MH33', 'MH36', 'MH18', 'MH22', 'MH23', 'MH43',
       'MH45', 'BR51', 'MH02', 'MH07', 'MH10', 'MH14', 'BR24', 'BR31',
       'BR38', 'BR45', 'BR06', 'BR07', 'BR08', 'BR11', 'BR12', 'BR18',
       'BR22', 'BR23', 'MH21'], dtype=object)